In [1]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import pandas as pd

In [54]:
N = 5557
CURRENT_MULTIPLIER= 1000.0
I_load = [0.0, 0.01007 , 0.02108 , 0.03016 , 0.00405, 0.00357 ]
FOLDER_NAME = 'G:\\My Drive\\Gdrive\\startup\\Novus Sentry\\data\\Data_load'

In [55]:
def read_all_files_in_folder_load(folder_name: str):
  path = folder_name
  file_list = [f for f in listdir(path) if isfile(join(path, f)) and f.endswith(".csv")]
  df_list = []
  for fname in file_list:
    print(fname)
    df = pd.read_csv(fname, header=None)
    df = df.dropna(axis=1, how='all')
    df_list.append(df)
  return df_list

In [56]:
def create_input_output_data(df_list):
  input_data = []
  for df in df_list:
    N = len(df.index)
    first_negative_current = df.iloc[:,1].argmin()
    print(first_negative_current, N, df.iloc[:,1].min())

    df.iloc[:, 1] /= 100.0
    df.iloc[:,1] = df.iloc[:,1].rolling(400).max()
    df = df.dropna()
    df = df.iloc[first_negative_current:,]
    df = df.reset_index(drop=True)

    input_data.append(df)
  return input_data

In [57]:
def insert_I_neg(I_neg):
  def current_sigmoidal_1(t, A, B, C, D, I_sd):
    I_measured = I_neg*(1.0-D*np.exp(-1.0*C/t)) -2*(I_neg*(1.0-D))*(0.5-1.0/(1.0+(np.exp(t/A)**B))) + I_sd
    return I_measured
  return current_sigmoidal_1

In [58]:
def current_sigmoidal_2(t, A, B, C, D, I_neg, I_sd):
  I_measured = I_neg*(1.0-D*np.exp(-1.0*C/t)) -2*(I_neg*(1.0-D))*(0.5-1.0/(1.0+np.exp((t/A)**B))) - I_sd
  return I_measured

In [59]:
def current_sigmoidal_3(t, A, B, C, D, I_neg, I_sd):
  I_measured = (I_neg-I_sd)*(1.0-D*np.exp(-1.0*C/t)) -2*((I_neg+I_sd)*(1.0-D))*(0.5-1.0/(1.0+np.exp((t/A)**B))) - I_sd
  return I_measured  

In [62]:
def fit(input_list):
  for i, input in enumerate(input_list):
    input.iloc[:,1] = input.iloc[:,1] - I_load[i]

    #N = input.iloc[:,1][input.iloc[:,1]<=0].tail(1).index.item()
    #N = (input.iloc[:,1]).abs().argsort()[:1].item()
    if input.iloc[-1,1] < 0.0:
        czcp = len(input.index)
    else:
        czcp = (input.iloc[:,1]).abs().argsort()[:1].item()
    print(czcp, I_load[i])

    i_measured = input.iloc[:czcp,1]*CURRENT_MULTIPLIER
    
    t = input.iloc[:czcp,0] -  input.iloc[0,0]
    t.iloc[0] = 0.001 
    t_all = input.iloc[:,0] - input.iloc[0,0] 

    I_neg = i_measured.min()
    #print(I_neg)
    f= current_sigmoidal_3
    parameters, covariance = curve_fit(f,t, i_measured, 
                                        p0=[100.0, 0.6, 0.14, 0.9, I_neg, -0.00033*CURRENT_MULTIPLIER], 
                                        bounds=([80.0,0.01,0.01,0.01,-np.inf,-0.005*CURRENT_MULTIPLIER],
                                                [500.0,1.0,1.0,1.0,-0.0001*CURRENT_MULTIPLIER,-0.000001*CURRENT_MULTIPLIER]),
                                        ftol=1e-12, verbose=0, max_nfev = 5000)

    # plt.scatter(t, i_measured,color='b',s=1)
    # plt.plot(t, f(t,*parameters), color='r')
    # plt.show()

    # plt.scatter(t_all, (input.iloc[:,1])*CURRENT_MULTIPLIER,color='b',s=1)
    # plt.plot(t_all, f(t_all,*parameters), color='r')
    # plt.show()
    #print(parameters[-1], (input.iloc[-1000:,1].mean())*CURRENT_MULTIPLIER, I_load[i])
  return parameters, covariance
  

In [63]:
if __name__=="__main__":
  df_list = read_all_files_in_folder_load(FOLDER_NAME)
  input = create_input_output_data(df_list)
  p, c = fit(input)


2022-10-26_074117095_Experiment 2 no load.csv
2022-10-26_210606095_Experiment 3.csv
2022-10-27_082959160_Experiment 4.csv
2022-10-27_171949981_DC Voltage(V).csv
2022-10-29_121517047_DC Voltage(V).csv
2022-10-31_125204212_DC Voltage(V).csv
49 112852 -0.507419938
4 51200 -3.3013862e-07
84 75952 1.89305368
66 54358 2.77472373
24 177880 -0.191976562
22 158842 -0.231243824
56451 0.0
17496 0.01007
24957 0.02108
53893 0.03016
40471 0.00405
45270 0.00357


In [45]:
input[5]

,0,1
0,172.385305,-0.004225
1,172.790103,-0.004225
2,173.200873,-0.004225
3,173.610988,-0.004225
4,174.014003,-0.004225
...,...,...
158416,65419.523176,0.000361
158417,65419.934143,0.000361
158418,65420.336479,0.000361
158419,65420.738640,0.000361
